In [1]:
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

[Insertion Sort List](https://leetcode.com/problems/insertion-sort-list/)。在链表上实现插入排序。

思路：使用一个新链表来保存有序链表，那么移动一个节点需要维护新链表的前驱位置与旧链表的后继位置。

In [2]:
def insertionSortList(head: ListNode) -> ListNode:
    if not head or not head.next:
        return head

    dummy = ListNode(-0x80000000)
    idx = head

    while idx:
        # 扫描寻找插入位置
        pre = dummy
        post = idx.next    # 原链表的后继
        while pre.next and pre.next.val <= idx.val:
            pre = pre.next
        pre.next, idx.next = idx, pre.next
        idx = post

    return dummy.next

[Remove Linked List Elements](https://leetcode.com/problems/remove-linked-list-elements/)。给一链表与一个key，删除所有值为key的节点。

思路：基本操作

In [ ]:
def removeElements(head: ListNode, val: int) -> ListNode:
    if not head:
        return head

    dummy = ListNode(None)
    dummy.next = head
    pre = dummy

    while pre and pre.next:
        if pre.next.val == val:
            pre.next = pre.next.next
        else:
            pre = pre.next

    return dummy.next

[Add Two Numbers II](https://leetcode.com/problems/add-two-numbers-ii/)。给两链表，每个节点表示一位数，每个链表表示一个数。对两链表求和。

思路：因为数字加和是从最低位开始加的，所以可以用栈。首先建立两个栈将两链表的所有值全压进去，然后依次取值进行相加。同时还要维护一个位数的指针$k$，高位加和还需要乘上$10$的$k$次方。最后将总和转成字符串便于处理。

In [13]:
def addTwoNumbers(l1: ListNode, l2: ListNode) -> ListNode:
    # 1. 边界条件
    if not l1:
        return l2
    if not l2:
        return l1

    # 2. 双链表入双栈
    s1, s2 = list(), list()
    idx = l1
    while idx:
        s1.append(idx.val)
        idx = idx.next
    idx = l2
    while idx:
        s2.append(idx.val)
        idx = idx.next

    # 3. 求和
    res = 0
    bit = 0    # 记录加到哪一位了
    while s1 and s2:
        cur_sum = s1.pop()+s2.pop()
        res += cur_sum*(10**bit)
        bit += 1

    while s1:
        res += s1.pop()*(10**bit)
        bit += 1
    while s2:
        res += s2.pop()*(10**bit)
        bit += 1

    # 4. 构建返回链表
    res = str(res)
    dummy = ListNode(None)
    idx = dummy
    while res:
        idx.next = ListNode(int(res[0]))
        res = res[1:]
        idx = idx.next

    return dummy.next

1

[Split Linked List in Parts](https://leetcode.com/problems/split-linked-list-in-parts/)。给一链表与$k$个桶，要求把链表尽量均分到$k$个桶中，

思路：$2$-pass，第一趟首先计算出链表的长度；第二趟开始做划分。若链表长度为$n$，那么每一个桶至少能分到$n//k$个节点，余数为$n\%k$，即前$n\%k$个桶都能额外分到一个节点。

In [16]:
def splitListToParts(root: ListNode, k: int):
    res = [None for _ in range(k)]

    # 计算链表长度
    idx = root
    n = 0
    while idx:
        n += 1
        idx = idx.next

    # 每个桶的基数与所有余数
    base, ex = divmod(n, k)
    idx = root    # 原链表的指针
    for i in range(k):
        sub_idx = res[i]    # 子链表的指针
        if ex:    # 如果有余数，优先加到前面的桶
            if sub_idx:
                sub_idx.next = ListNode(idx.val)
                sub_idx = sub_idx.next
            else:
                res[i] = ListNode(idx.val)
                sub_idx = res[i]
            idx = idx.next
            ex -= 1
        for j in range(base):
            if sub_idx:
                sub_idx.next = ListNode(idx.val)
                sub_idx = sub_idx.next
            else:
                res[i] = ListNode(idx.val)
                sub_idx = res[i]
            idx = idx.next
    return res

[<__main__.ListNode at 0x1f1787ae208>, <__main__.ListNode at 0x1f1787aed68>]

[Flatten a Multilevel Doubly Linked List](https://leetcode.com/problems/flatten-a-multilevel-doubly-linked-list/)。一双向链表，除了有前驱节点指针与后继结点指针外，还有一个child指针，指向另一个双向链表的首节点。这样的节点构造构成了一个有层次的多链表结构。现要求把多层结构展开成单层结构，当存在child指针时，优先连接子链表。

思路：从首节点开始扫描，当遇到既有后继又有child的节点时，将后继节点入栈即可。注意是双向链表，入栈时需要完全断链。总共有四种情况，无next无child、有next无child、有child无next、无next无child。

In [ ]:
class Node:
    def __init__(self, val, prev, next, child):
        self.val = val
        self.prev = prev
        self.next = next
        self.child = child


def flatten(head: 'Node') -> 'Node':
    if not head:
        return head

    s = list()
    idx = head

    while idx.next or idx.child or s:
        if not idx.next and not idx.child:    # 无next无child
            next_node = s.pop()
            idx.next = next_node
            next_node.prev = idx
            idx = idx.next
        elif not idx.child and idx.next:    # 有next无child
            idx = idx.next
        else:
            if idx.next:    # 有next有child
                s.append(idx.next)
                idx.next.prev = None
            idx.next = idx.child
            idx.child.prev = idx
            idx.child = None
            idx = idx.next

    return head

[Linked List Components](https://leetcode.com/problems/linked-list-components/)。给一个链表，每个节点的值都不一样；在给一数组，每个元素都是链表中的节点值，判断这些值在链表中构成了几个相连的部分。

思路：找分部的尾节点即可。扫描链表，如果当前节点不在G中，则跳过；若当前节点在G中但是下一个节点不在G中，说明是一个分部的尾节点。

In [ ]:
def numComponents(head: ListNode, G) -> int:
    G = set(G)
    idx = head
    res = 0

    while idx:
        if idx.val in G:
            if idx.next and idx.next.val not in G:
                res += 1
            if not idx.next:    # 末尾
                res += 1

        idx = idx.next

    return res